1. create a spark dataframe without using a csv reader 
2. Make sure each column is the correct type
3. handle NA values

In [1]:
from pyspark.sql import SQLContext, Row,DataFrame
from pyspark.sql.types import *
filename = 'airquality.csv'
# filename = 's3://adcoregon/Joanne/data/airquality.csv' -- this is how you 
rdd = sc.textFile(filename)

# rdd.map(lambda x: if x=='NA')
# rdd.collect()[0].split(',')

rdd2=rdd.map(lambda x: x.split(','))
header = rdd2.first()
rdd_noHeader=rdd2.filter(lambda x: x != header)


#infer type based on what is there
def infer_type(x):
    if '"' in x:
        x = x.replace('"','')
        
    if '.' in x:
        return float(x)
    elif x =='NA':
        return None
    else:
        return int(x) 
        
cleaned = rdd_noHeader.map(lambda x: [infer_type(a) for a in x])
header_stripped = [x.replace('"','') for x in header]

In [2]:
# converting to DatFrame w/o Schema
df = cleaned.toDF(header_stripped)
df.show()
print df

+---+-----+-------+----+----+-----+---+
|   |Ozone|Solar.R|Wind|Temp|Month|Day|
+---+-----+-------+----+----+-----+---+
|  1|   41|    190| 7.4|  67|    5|  1|
|  2|   36|    118|null|  72|    5|  2|
|  3|   12|    149|12.6|  74|    5|  3|
|  4|   18|    313|11.5|  62|    5|  4|
|  5| null|   null|14.3|  56|    5|  5|
|  6|   28|   null|14.9|  66|    5|  6|
|  7|   23|    299| 8.6|  65|    5|  7|
|  8|   19|     99|13.8|  59|    5|  8|
|  9|    8|     19|20.1|  61|    5|  9|
| 10| null|    194| 8.6|  69|    5| 10|
| 11|    7|   null| 6.9|  74|    5| 11|
| 12|   16|    256| 9.7|  69|    5| 12|
| 13|   11|    290| 9.2|  66|    5| 13|
| 14|   14|    274|10.9|  68|    5| 14|
| 15|   18|     65|13.2|  58|    5| 15|
| 16|   14|    334|11.5|  64|    5| 16|
| 17|   34|    307|null|  66|    5| 17|
| 18|    6|     78|18.4|  57|    5| 18|
| 19|   30|    322|11.5|  68|    5| 19|
| 20|   11|     44| 9.7|  62|    5| 20|
+---+-----+-------+----+----+-----+---+
only showing top 20 rows

DataFrame[: bi

In [46]:
# alternatively, we could impose a schema
schema_ = StructType([StructField('index',IntegerType(),True),\
           StructField(header_stripped[1],IntegerType(),True),\
           StructField(header_stripped[2],IntegerType(),True),\
           StructField(header_stripped[3],DoubleType(),True),\
           StructField(header_stripped[4],IntegerType(),True),\
           StructField(header_stripped[5],IntegerType(),True),\
           StructField(header_stripped[6],IntegerType(),True)])

# shorthand:
df = cleaned.toDF(schema=schema_)

# full:
df = sqlContext.createDataFrame(cleaned,schema_)
df.show()
print df.printSchema()
# df.collect()

+-----+-----+-------+----+----+-----+---+
|index|Ozone|Solar.R|Wind|Temp|Month|Day|
+-----+-----+-------+----+----+-----+---+
|    1|   41|    190| 7.4|  67|    5|  1|
|    2|   36|    118|null|  72|    5|  2|
|    3|   12|    149|12.6|  74|    5|  3|
|    4|   18|    313|11.5|  62|    5|  4|
|    5| null|   null|14.3|  56|    5|  5|
|    6|   28|   null|14.9|  66|    5|  6|
|    7|   23|    299| 8.6|  65|    5|  7|
|    8|   19|     99|13.8|  59|    5|  8|
|    9|    8|     19|20.1|  61|    5|  9|
|   10| null|    194| 8.6|  69|    5| 10|
|   11|    7|   null| 6.9|  74|    5| 11|
|   12|   16|    256| 9.7|  69|    5| 12|
|   13|   11|    290| 9.2|  66|    5| 13|
|   14|   14|    274|10.9|  68|    5| 14|
|   15|   18|     65|13.2|  58|    5| 15|
|   16|   14|    334|11.5|  64|    5| 16|
|   17|   34|    307|null|  66|    5| 17|
|   18|    6|     78|18.4|  57|    5| 18|
|   19|   30|    322|11.5|  68|    5| 19|
|   20|   11|     44| 9.7|  62|    5| 20|
+-----+-----+-------+----+----+---

In [6]:
# register the table 
df.registerTempTable('mytable')

# query 
sqlContext.sql('select * from mytable where Month = 6').show()

+---+-----+-------+----+----+-----+---+
|   |Ozone|Solar.R|Wind|Temp|Month|Day|
+---+-----+-------+----+----+-----+---+
| 32| null|    286| 8.6|  78|    6|  1|
| 33| null|    287| 9.7|  74|    6|  2|
| 34| null|    242|16.1|  67|    6|  3|
| 35| null|    186| 9.2|  84|    6|  4|
| 36| null|    220| 8.6|  85|    6|  5|
| 37| null|    264|14.3|  79|    6|  6|
| 38|   29|    127| 9.7|  82|    6|  7|
| 39| null|    273| 6.9|  87|    6|  8|
| 40|   71|    291|13.8|  90|    6|  9|
| 41|   39|    323|11.5|  87|    6| 10|
| 42| null|    259|10.9|  93|    6| 11|
| 43| null|    250| 9.2|  92|    6| 12|
| 44|   23|    148|null|  82|    6| 13|
| 45| null|    332|13.8|  80|    6| 14|
| 46| null|    322|11.5|  79|    6| 15|
| 47|   21|    191|14.9|  77|    6| 16|
| 48|   37|    284|20.7|  72|    6| 17|
| 49|   20|     37| 9.2|  65|    6| 18|
| 50|   12|    120|11.5|  73|    6| 19|
| 51|   13|    137|10.3|  76|    6| 20|
+---+-----+-------+----+----+-----+---+
only showing top 20 rows

